#### Objective
We've developed a new page and we want to now check the purchase conversion. We split our users into 2 evenly groups:
1. Control : They get the old webpage
2. Treatment : They get the new webpage

Purchase convertion = Converted users/Exposed users

We have 3 weeks of logged exposure/conversion data.

Exposure: A user is bucketed as control or treatment and sees their corresponding page for the first time in the experiment duration.
Conversion: An exposed user makes a purchase within 7 days of being first exposed.


In [2]:
import pandas as pd
import numpy as np
import datetime
from scipy.stats import chi2_contingency, beta
from IPython.display import Image

In [4]:
df = pd.read_csv('ab_data.csv')

In [7]:
df.sample(10)

,user_id,timestamp,group,landing_page,converted
61304,843747,2017-01-02 22:06:42.535236,control,old_page,0
144268,646784,2017-01-03 21:16:57.066297,control,old_page,0
218427,780056,2017-01-22 10:47:10.358707,treatment,new_page,0
179681,855297,2017-01-03 14:30:53.427833,control,old_page,0
204566,668908,2017-01-03 17:24:13.891761,control,old_page,0
30855,777104,2017-01-12 00:39:43.727759,treatment,new_page,0
60707,671423,2017-01-04 13:42:08.000207,treatment,new_page,1
194465,680872,2017-01-23 19:22:04.634219,treatment,new_page,0
174254,909228,2017-01-18 02:36:38.689952,treatment,new_page,0
63748,722720,2017-01-24 04:26:52.474494,treatment,new_page,0


Each row is logged when user is exposed to a webpage.

timestamp: time the user is first exposed
group: our defined bucket
landing_page: Which page are they seeing
converted: Initialized to 0. Changes to 1 if the user makes a purchase within 7 days of first exposure

In [8]:
start_time = datetime.datetime.strptime(df['timestamp'].min(), '%Y-%m-%d %H:%M:%S.%f')
end_time = datetime.datetime.strptime(df['timestamp'].max(), '%Y-%m-%d %H:%M:%S.%f')
data_duration = (end_time - start_time).days

print(f"Number of unique users in experiment: {df['user_id'].nunique()}")
print(f"Data collected for {data_duration} days")
print(f"Landing pages to compare: {df['landing_page'].unique().tolist()}")
print(f"Percentage of users in control: {round(df[df['group']=='control'].shape[0] * 100 / df.shape[0])}%")

Number of unique users in experiment: 290584
Data collected for 21 days
Landing pages to compare: ['old_page', 'new_page']
Percentage of users in control: 50%


In [9]:
# Checking for any discrepancies
counter = df['user_id'].value_counts()
(counter > 1).value_counts()

False    286690
True       3894
Name: user_id, dtype: int64

There are 3894 users who are exposed to both webpages. Hence, removing these users to get a better sample.

In [10]:
valid_users = pd.DataFrame(counter[counter == 1].index, columns=['user_id'])
df = df.merge(valid_users, on=['user_id'])

In [13]:
# Add week column to see the data as you would during experiment
df['week'] = df['timestamp'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f').isocalendar()[1])
df.sample(10)

,user_id,timestamp,group,landing_page,converted,week
65345,874857,2017-01-19 14:08:51.004368,control,old_page,0,3
211070,704096,2017-01-10 17:37:15.656560,control,old_page,0,2
238122,932933,2017-01-08 07:45:00.804054,control,old_page,0,1
45973,768739,2017-01-23 11:29:53.253414,treatment,new_page,0,4
3119,910814,2017-01-23 07:55:31.241227,treatment,new_page,0,4
112167,767853,2017-01-22 20:48:17.851837,treatment,new_page,0,3
50918,938175,2017-01-17 21:32:21.323149,treatment,new_page,0,3
140532,861618,2017-01-13 03:24:55.969973,control,old_page,0,2
218627,883300,2017-01-16 13:42:09.410523,control,old_page,0,3
43330,817931,2017-01-17 06:20:46.352723,control,old_page,0,3


In [14]:
df['week'].value_counts()

2    91380
3    91056
1    83745
4    20509
Name: week, dtype: int64

In [15]:
# Get Stats across all weeks
NUM_WEEKS = 4
experiment_data = df[df['week'] <= NUM_WEEKS]
control = experiment_data[experiment_data['group']=='control']
treatment = experiment_data[experiment_data['group']=='treatment']

control_conversion_perc = round(control['converted'].sum() * 100/ control['converted'].count(), 3)
treatment_conversion_perc = round(treatment['converted'].sum() * 100/ treatment['converted'].count(), 3)
lift = round(treatment_conversion_perc - control_conversion_perc, 3)

print(f"Treatment Conversion Rate: {treatment_conversion_perc}%")
print(f"Control Conversion Rate: {control_conversion_perc}%")
print(f"Lift = {lift}%")

Treatment Conversion Rate: 11.873%
Control Conversion Rate: 12.017%
Lift = -0.144%


#### Using Chi-Squared Test

Null hypothesis : Control and treatment are independent <br />
Alternative hypothesis : Control and treatment are not independent

In [16]:
# Create Contingency Table for Chi Squared Test
control_converted = control['converted'].sum()
treatment_converted = treatment['converted'].sum()
control_non_converted = control['converted'].count() - control_converted
treatment_non_converted = treatment['converted'].count() - treatment_converted
contingency_table = np.array([[control_converted, control_non_converted], 
                              [treatment_converted, treatment_non_converted]])

In [17]:
contingency_table

array([[ 17220, 126073],
       [ 17025, 126372]], dtype=int64)

In [18]:
chi, p_value, _, _ = chi2_contingency(contingency_table, correction=False)

In [19]:
chi, p_value

(1.426794609399621, 0.23228827305833816)

Since the p_value > 0.05, we cannot reject null hypothesis. Hence, we cannot conclude if there exists a relationship between the control and treatment groups.

In [20]:
print(f"{round(p_value * 100, 2)}% probability that a more extreme chi square than {round(chi, 3)} would have occured by chance")

23.23% probability that a more extreme chi square than 1.427 would have occured by chance


But this is tough to interpret. We would to say something about the actual maginitude of lift. Something like this:

In [22]:
print(f"(We CANNOT say this) We are {round(p_value * 100, 2)}% confident that our lift = {lift}%")

(We CANNOT say this) We are 23.23% confident that our lift = -0.144%


#### Bayesian Approach

We want to input the prior distribution and have the experiment update the parameters to create posterier distributions. Since these prior & posterior distributions will be used to sample Conversion Rate, we model them after beta distribution. </br>

Let's create the prior beta distribtion from the first weeks of conversion data

In [23]:
prior = df[(df['week'] == 1) & (df['group']=='control')]

In [24]:
prior_means = []
for i in range(10000):
    prior_means.append(prior.sample(1000)['converted'].mean())

In [25]:
prior_means[:10]

[0.125, 0.112, 0.119, 0.118, 0.122, 0.141, 0.117, 0.122, 0.123, 0.108]

In [26]:
# Model Beta Distribtion from sample means
prior_alpha, prior_beta, _, _ = beta.fit(prior_means, floc=0, fscale=1)

In [27]:
# Get Stats
NUM_WEEKS = 4 # Vary number to get experiment data at weekly points in time
experiment_data = df[(df['week'] > 1) & (df['week'] <= NUM_WEEKS)]
control = experiment_data[experiment_data['group']=='control']
treatment = experiment_data[experiment_data['group']=='treatment']

control_conversion_perc = round(control['converted'].sum() * 100/ control['converted'].count(), 3)
treatment_conversion_perc = round(treatment['converted'].sum() * 100/ treatment['converted'].count(), 3)
lift = round((treatment_conversion_perc - control_conversion_perc) / control_conversion_perc , 3)

print(f"Treatment Conversion Rate: {treatment_conversion_perc}%")
print(f"Control Conversion Rate: {control_conversion_perc}%")
print(f"Lift = {lift}%")

Treatment Conversion Rate: 11.909%
Control Conversion Rate: 12.058%
Lift = -0.012%


In [28]:
control_converted = control['converted'].sum()
treatment_converted = treatment['converted'].sum()
control_non_converted = control['converted'].count() - control_converted
treatment_non_converted = treatment['converted'].count() - treatment_converted

# Update Prior parameters with experiment conversion rates
posterior_control = beta(prior_alpha + control_converted, prior_beta + control_non_converted)
posterior_treatment = beta(prior_alpha + treatment_converted, prior_beta + treatment_non_converted)

# Sample from Posteriors
control_samples = posterior_control.rvs(1000)
treatment_samples = posterior_treatment.rvs(1000)
probability = np.mean(treatment_samples > control_samples)
print(f"Probability that treatment > control: {probability * 100}%")

Probability that treatment > control: 16.5%


In [29]:
(control_mu), (control_var) = posterior_control.stats()
(treatment_mu), (treatment_var) = posterior_treatment.stats()
print(f"Control Posterior: Mean: {control_mu}, Variance: {control_var}") 
print(f"Treatment Posterior: Mean: {treatment_mu}, Variance: {treatment_var}") 

Control Posterior: Mean: 0.120562727716921, Variance: 1.03362049955054e-06
Treatment Posterior: Mean: 0.11909376960818509, Variance: 1.0245199310611838e-06


In [30]:
lift_percentage = (treatment_samples - control_samples) / control_samples
print(f"Probability that we are seeing a 2% lift: {np.mean((100 * lift_percentage) > 2) * 100}%")

Probability that we are seeing a 2% lift: 0.5%


For our given case, control group is performing better than the treatment group.

Advantages of Bayesian over Frequentist:

Results are more interpretable than the ones we got from the frequentist approach
